In [1]:
!pip install galois
!pip install scipy

/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found
/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found


In [1]:
import galois
import numpy as np
import time
import os
from scipy import stats
import csv

In [2]:
field = galois.GF(7)
#definisco una matrice k x (n-k)
k = 7 #le righe
n_k = 7 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 visto da pdf myLESS_CF pagina 38 di 45

In [3]:
max_column = 5
#number of iteration for Montecarlo simulatiom
num_iterations = 1000

In [2]:
#FOR IMPLEMENTS CF 4.4
def has_duplicate_rows(matrix):
    _, counts = np.unique(matrix, axis=0, return_counts=True)
    return np.any(counts > 1)

def make_bin_matrix(dim, field):
    bin_matrix = np.zeros(dim, dtype=int)
    perm = np.random.permutation(dim[0])
    for i in range(dim[0]):
        bin_matrix[i, perm[i]] = 1
    return field(bin_matrix)

#initial matrix x matrix with s or s^(-1)
def make_matrix_xS(matrix, field_order):
    start = time.perf_counter() #this is for total time
    matrix_S = field(np.zeros((k, k), dtype = int))
    for i, row in enumerate(matrix):
        #print(row)
        if np.all(row == row[0]):
            if row[0] != 0:
                row[:] = 1
                matrix_S[i, i] = 1
            else:
                matrix_S[i, i] = 1
        else:
        #print("Type row: ", type(row)) IS GALOIS (CONFIRMED)
            s = np.sum(row)
            s_prime = np.sum(row ** (field_order-2))
            #print("s = ", s)
            #print("s' = ", s_prime)
            if s != 0:
                matrix_S[i,i] = np.reciprocal(s)
            elif s_prime != 0:
                matrix_S[i,i] = s_prime
            else:
                #OVVIAMENTE NELLA FORMA 5 AVENDO UNA MATRICE CHE HA TUTTI 1 MI ENTRA SEMPRE QUA (è stato risolto poichè c'è
                # un controllo su quei tipi di righe, e si va mettere un 1 sulla riga della matrice S, fatto negli if sopra)
                end = time.perf_counter() #this is for total time
                duration = (end - start) * 1000 #this is for total time
                print("esco")
                return [], duration #this is for total time
    end = time.perf_counter() #this is for total time
    duration = (end - start) * 1000 #this is for total time
    #print(f"Matrice s\n", matrix_S)
    matrix = matrix_S @ matrix
    #print(f"Matrix\n", matrix)
    return matrix, duration #this is for total time

# order by count number of 0, 1, ..., q in a vector (dictionary). A multiset1 < multiset2 if number of 0 > number of 0 of 
# second multiset. If the number of 0 is the same check number of 1 and so on
def order_row(matrix, error_counter):  
    if len(matrix) == 0:
        #no error_counter += 1 because isn t an error of algoritm
        return [], 0, 0, [], 0, 0, error_counter
    start = time.perf_counter()#this is for make file csv dati_differences_upd2 and only count/ord
    vector_of_counts = np.zeros((matrix.shape[0], field.order + 1), dtype = int)
    n_colonne = matrix.shape[1]  
    #start = time.perf_counter()
    
    #for row in matrix:
    for i, row in enumerate(matrix):
        #bp.bincout count frequency of element in row
        vector_of_counts[i] = np.bincount(row, minlength = field.order + 1)
    end = time.perf_counter() #these 2 for file only count/ord
    duration = (end - start) * 1000
    #vector_of_counts = np.array(vector_of_counts)
    if has_duplicate_rows(vector_of_counts):
        error_counter += 1
        #print(vector_of_counts)
        print("Duplicated rows all columns",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    
    sorted_index = np.lexsort([vector_of_counts[:, i] for i in range(n_colonne, -1, -1)])[::-1]
    #end = time.perf_counter() #these 2 for make file csv dati_differences_upd2
    #duration = (end - start) * 1000    
    
    start = time.perf_counter() #this is for make file csv dati_differences_upd2 and only count/ord
    vector_of_counts1 = np.zeros((matrix.shape[0], max_column), dtype = int)
    for i, row in enumerate(matrix):
        #BY REMOVING THIS FILTERING (BELOW) WHICH WOULD ADD OVERHEAD AS I HAVE TO SCROLL THROUGH THE ENTIRE VECTOR REGARDLESS
        #I WILL SAVE IN TERMS OF TIME
        #valid_values = row[row < max_column]
        vector_of_counts1[i] = np.bincount(row, minlength = max_column)[:max_column]
    #print(vector_of_counts1)
    end = time.perf_counter() #these 2 for file only count/ord
    duration1 = (end - start) * 1000
    if has_duplicate_rows(vector_of_counts1):
        error_counter += 1
        #print(vector_of_counts1)
        print("Duplicated rows less columns",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    sorted_index1 = np.lexsort([vector_of_counts1[:, i] for i in range(vector_of_counts1.shape[1]-1, -1, -1)])[::-1]
    
    """for raw in matrix[sorted_index1]:
        print(np.sort(raw))"""
    #print()
    #end = time.perf_counter() #these 2 for make file csv dati_differences_upd2
    #duration1 = (end - start) * 1000 
    print(np.array_equal(matrix[sorted_index], matrix[sorted_index1]))
    if np.array_equal(matrix[sorted_index], matrix[sorted_index1]):
        print("If positivo")
        return matrix[sorted_index], duration, 0, matrix[sorted_index1], duration1, 0, error_counter
    else:
        print("If negativo")
        error_counter += 1
        print(error_counter)
        return [], duration, 0, [], duration1, 0, error_counter

# order column as always
def sort_by_column(matrix, rows, num_swap_columns):
    #print("Funzione ordina per colonna")
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    initial_position = np.arange(matrix.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe, se c'è conflitto passa a el seconda riga
    sorting_indices = np.lexsort([matrix[row, :] for row in rows[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, index in enumerate(sorting_indices):
        if index != initial_position[i]:
            num_swap_columns += 1
    
    #riordina la matrice in base agli indici trovati
    sorted_matrix = matrix[:, sorting_indices]
    """for raw in sorted_matrix:
        print(raw)
    print()"""
    return sorted_matrix, num_swap_columns


In [3]:
def check_non_zero_row(row):
    #return true if all elements != 0
    return np.all(row)

#make scaling matrix for every row with all elements != 0 and A x D added to a vector
def scaled_matrices(matrix):
    scaled_matrices = []
    scaling_matrix = field(np.zeros((k, n_k), dtype=int))
    for i, row in enumerate(matrix):
        if check_non_zero_row(row):
            for j in range(n_k):
                scaling_matrix[j,j] = np.reciprocal(matrix[i,j])
            #print(f"Matrice per scalare\n", scaling_matrix)
            #print("Matrix x scalata")
            #print(matrix @ scaling_matrix)
            scaled_matrices.append(matrix @ scaling_matrix)

    return scaled_matrices

#aggiusta sta cazzo di cosa vedi come deve funzionare considerando che ho una lista di matrici e mi devo portare appresso
#i tempi e quant'altro. IL PROBLEMA QUA È CAPIRE COME GESTIRE IL TUTTO NEL SENSO CHE IO DA OGNI MATRICE POTREI RICAVARE
#POTENZIALMENTE 127 ALTRE MATRICI E APPLICARE QUINDI LA CF4 ANCORA PIÙ VOLTE PER CUI POTREI FAR SALIRE IL NUMERO DEI FALLIMNETI
#MENTRE POI VADO A CONTARE SOLO 1000 ITERAZIONI. qUINDI CAPISCI COME CONTARE L'ERROR_COUNTER E SE UN ALGORITMO FALLISCE
#SULLA MATRICE DI UNA RIGA SI DEVE FERMARE TUTTO SU QUELLA MATRICE OPPURE VADO AVANTI ALLA PROSSIMA RIGA.
def apply_cf_4_4(scaled_matrices, error_counter, add_to_iteration):
    all_matrices = []
    num_swap_col = 0
    if len(scaled_matrices) != 0:
        for matrix in scaled_matrices:
            add_to_iteration += 1
            matrix, dur = make_matrix_xS(matrix, field.order)
            row_sort, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter = order_row(matrix, error_counter)
            if not len(row_sort) == 0:
                scaled_matrix_sorted, num_swap_col = sort_by_column(row_sort, range(k), 0)
                #print(scaled_matrix_sorted)
                all_matrices.append(scaled_matrix_sorted)
            else: #there when i go in error not do error_counter += 1 because i do this in other functions
                return False, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter, [], add_to_iteration
        return True, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter, all_matrices, add_to_iteration
    else: 
        return False, 0, 0, 0, 0, 0, error_counter, [], add_to_iteration


def cf5(all_matrices, error_counter):
    if len(all_matrices) != 0:
        smallest_matrix = np.zeros((len(all_matrices), max_column), dtype = int)
        for i, matrix in enumerate(all_matrices):
            smallest_matrix[i] = np.bincount(matrix[0,:], minlength = max_column)[:max_column]

        #print("This is frequence of every first line", smallest_matrix, smallest_matrix.shape) 

        if (smallest_matrix.shape[0] > 1): 
            smallest_index = np.lexsort([smallest_matrix[:, i] for i in range(smallest_matrix.shape[1]-1, -1, -1)])[::-1]
            #the row below is very rare with big dimensions. To make an error it is sufficient that the first 2 rows are equal
            if (np.array_equal(smallest_matrix[smallest_index[0]], smallest_matrix[smallest_index[1]])):
                #print("Duplicated rows in frequency first line")
                error_counter += 1
                return [], error_counter #return empty element for make a check in other funcrtion
            else:
                return all_matrices[smallest_index[0]], error_counter
        else:
            return all_matrices[0], error_counter
    else:
        return [], error_counter


In [47]:
error_counter = 0
A = field(np.random.randint(0, field.order, size=(k, n_k))) 
while(has_duplicate_rows(A)):
    A = field(np.random.randint(0, field.order, size=(k, n_k)))
print(f"Questa è quella di partenza\n", A)
matrici_scalate = scaled_matrices(A)
esito, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter, all_matrices = apply_cf_4_4(matrici_scalate, error_counter)
smallest, error_counter = cf5(all_matrices, error_counter)
print(f"Smallest matrix --> CF5 = \n", smallest)

Questa è quella di partenza
 [[3 2 3 0 6 6 1]
 [5 5 0 6 2 5 3]
 [6 3 1 4 3 5 5]
 [1 6 4 3 3 0 6]
 [6 3 2 6 4 1 4]
 [2 5 0 3 4 0 6]
 [0 1 2 6 1 5 0]]
Matrice per scalare
 [[6 0 0 0 0 0 0]
 [0 5 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 2 0 0 0]
 [0 0 0 0 5 0 0]
 [0 0 0 0 0 3 0]
 [0 0 0 0 0 0 3]]
Matrix x scalata
[[4 3 3 0 2 4 3]
 [2 4 0 5 3 1 2]
 [1 1 1 1 1 1 1]
 [6 2 4 6 1 0 4]
 [1 1 2 5 6 3 5]
 [5 4 0 6 6 0 4]
 [0 5 2 5 5 1 0]]
Matrice per scalare
 [[6 0 0 0 0 0 0]
 [0 5 0 0 0 0 0]
 [0 0 4 0 0 0 0]
 [0 0 0 6 0 0 0]
 [0 0 0 0 2 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 2]]
Matrix x scalata
[[4 3 5 0 5 6 2]
 [2 4 0 1 4 5 6]
 [1 1 4 3 6 5 3]
 [6 2 2 4 6 0 5]
 [1 1 1 1 1 1 1]
 [5 4 0 4 1 0 5]
 [0 5 1 1 2 5 0]]
True
If positivo
[[0 0 1 1 3 5 5]
 [2 4 0 3 0 3 3]
 [0 2 2 1 3 3 4]
 [5 0 3 6 3 4 1]
 [5 2 2 2 5 0 6]
 [1 1 1 1 1 1 1]
 [5 1 6 4 4 6 3]]
True
If positivo
[[0 0 1 1 3 5 5]
 [4 5 0 0 3 4 5]
 [5 0 2 6 4 4 1]
 [5 3 1 4 3 6 0]
 [0 4 5 3 5 4 1]
 [1 1 1 1 1 1 1]
 [6 2 4 5 3 4 5]]
This is frequence of 

In [4]:
#try a montecarlo simulation for check success perc
field = galois.GF(127)
#definisco una matrice k x (n-k)
k = 126 #le righe
n_k = 126 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 visto da pdf myLESS_CF pagina 38 di 45
max_column = 12
#number of iteration for Montecarlo simulatiom
num_iterations = 1000

def montecarlo_simulation(num_iterations):
    vector_operations = []
    vector_operations1 = []
    vector_duration1 = []
    vector_duration1_2 = []
    error_counter = 0
    add_to_iteration = 0
    duration1_accumulator = 0
    duration1_2_accumulator = 0
    operations_accumulator = 0
    operations1_accumulator = 0
    csv_file_path = '/home/michele/Desktop/Tirocinio_giusto/Tirocinio/post_uarta_revisione/dati_cf_4_5(2).csv'
    file_exists = os.path.exists(csv_file_path)
    #this is for total time
    #with open('dati_tot_time.csv', mode='a', newline='') as file:
    with open(csv_file_path, mode='a', newline='') as file:
        writer_csv = csv.writer(file)
        if not file_exists:
            writer_csv.writerow(['File','Iteration', 'Max_columns', 'Duration_w_all_Columns', 'Duration_w_max_columns', 'Num_op_w_all_columns', 'Num_op_w_max_columns'])

        for i in range(num_iterations):
            A = field(np.random.randint(0, field.order, size=(k, n_k))) 
            while(has_duplicate_rows(A)):
                A = field(np.random.randint(0, field.order, size=(k, n_k))) 
            
            A, dur_xS = make_matrix_xS(A,field.order)

            matrici_scalate = scaled_matrices(A)
            check, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter, all_matrices, add_to_iteration = apply_cf_4_4(matrici_scalate, error_counter, add_to_iteration)

            if check:
                smallest, error_counter = cf5(all_matrices, error_counter)
                duration1_accumulator += duration1_1
                duration1_2_accumulator += durations1_2
                operations_accumulator += operations
                operations1_accumulator += operations1
                vector_operations.append(operations)
                vector_operations1.append(operations1)
                vector_duration1.append(duration1_1)
                vector_duration1_2.append(durations1_2)
                print("Add iteration: ", add_to_iteration)
                if len(smallest) != 0:
                    writer_csv.writerow([6,i, max_column, duration1_1, durations1_2, operations, operations1])
                else:
                    print("Add iteration: ", add_to_iteration)
                    print("Error in cf function", error_counter) 
            else:
                print("Add iteration: ", add_to_iteration)
                print("Error in apply _cf_4_4", error_counter) #if error_counter not increments error in martrix_S
            
    t_stat1, p_value1 = stats.ttest_rel(vector_duration1, vector_duration1_2)
    print(f"T-statistica: {t_stat1}, P-value: {p_value1}")
    if p_value1 < 0.05:
        print("La differenza tra durate normali e sufficienti è statisticamente significativa.")
    else:
        print("La differenza non è significativa.")
            
    return smallest, duration1_accumulator, duration1_2_accumulator, operations_accumulator, operations1_accumulator, p_value1, (num_iterations-error_counter)/num_iterations


smallest, duration1_accumulator, duration1_2_accumulator, operations_accumulator, operations1_accumulator, p_value, good_perc = montecarlo_simulation(num_iterations)



True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
Duplicated rows less columns 1
Add iteration:  50
Error in apply _cf_4_4 1
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If

1


In [5]:
#print some stats
#this is for total time
#nome_file = "stat_without_sort_with_S_tot_time.txt"
nome_file = "stat_cf_4_5.txt"


with open(nome_file, 'a') as file:
    file.write(f"Qui c'è il limite colonne a: {max_column} file 6\n")
    file.write(f"Tempo esecuzione con tutte colonne: {duration1_accumulator} ms\n")
    file.write(f"Tempo esecuzione con {max_column} colonne: {duration1_2_accumulator} ms\n")
    file.write(f"Numero operazioni con tutte colonne: {operations_accumulator}\n")
    file.write(f"Numero operazioni con {max_column} colonne: {operations1_accumulator}\n")
    file.write(f"P-value per operazioni: {p_value}\n")
    file.write(f"Percentuale test positivi (tutte colonne = meno colonne): {good_perc}%\n\n")